# Lesson 3

In [1]:
import numpy as np
import matplotlib.pyplot as plt

Задание 1: Измените функцию calc_logloss так, чтобы нули по возможности не попадали в np.log.
Задание 2: Подберите аргументы функции eval_model для логистической регрессии таким образом, чтобы log loss был минимальным.
Задание 3: Создайте функцию calc_pred_proba, возвращающую предсказанную вероятность класса 1 (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred_proba).
Задание 4: Создайте функцию calc_pred, возвращающую предсказанный класс (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred).
Задание 5: Посчитайте Accuracy, матрицу ошибок, точность и полноту, а также F1 score.
Задание 6: Могла ли модель переобучиться? Почему? 

In [2]:
X = np.array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
              [1, 1, 2, 1, 3, 0, 5, 10, 1, 2],
              [500, 700, 750, 600, 1450,
               800, 1500, 2000, 450, 1000],
              [1, 1, 2, 1, 2, 
               1, 3, 3, 1, 2]], dtype = np.float64)

y = np.array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1], dtype = np.float64)

In [3]:
def calc_std_feat(x):
    res = (x - x.mean()) / x.std()
    return res

In [4]:
X_st = X.copy()
X_st[2, :] = calc_std_feat(X[2, :])

In [5]:
def sigmoid(z):
    res = 1 / (1 + np.exp(-z))
    return res

In [6]:
def calc_logloss(y, y_pred):
    for y in y_pred:
        if y == 0:
            y = 0.001
        elif y == 1:
            y = 0.999
    err = - np.mean(y * np.log(y_pred) + (1.0 - y) * np.log(1.0 - y_pred))
    return err

In [7]:
def eval_model(X, y, iterations, alpha=1e-4):
    np.random.seed(42)
    W = np.random.randn(X.shape[0])
    n = X.shape[1]
    for i in range(1, iterations+1):
        z = np.dot(W, X)
        y_pred = sigmoid(z)
        err = calc_logloss(y, y_pred)
        W -= alpha * (1/n * np.dot((y_pred - y), X.T))
    if i % (iterations / 10) == 0:
        print(i, W, err)
    return W

In [8]:
parameters_grid = [
    (0.1, 100000), (0.01, 100000), (0.001, 100000), (0.0001, 100000), (0.00001, 100000), (0.000001, 100000)
]

for learning_rate, n_iterations in parameters_grid:
    W = eval_model(X_st, y, iterations=n_iterations, alpha=learning_rate)

100000 [-35.8933164   -3.59426199  -9.64639296  29.4060647 ] 1.6045431002968396
100000 [-11.27192801  -1.45338517  -2.38300258   9.49385018] 1.0068531914908074
100000 [-2.77073816 -0.9958014   0.56651721  3.26761217] 0.858163939153129
100000 [-0.29755454 -0.72668938  1.06147096  1.3966418 ] 0.7462671576593806
100000 [ 0.25809285 -0.68188567  0.6883469   1.2412009 ] 0.6348708150530281
100000 [ 0.45886991 -0.25439684  0.64531306  1.46696013] 0.26791124065499605


In [9]:
def calc_pred_proba(W, X):
    y_pred_proba = 1 / (1 + np.exp(-np.dot(W, X)))
    return y_pred_proba

In [13]:
y_pred_proba = calc_pred_proba(eval_model(X_st, y, iterations=n_iterations, alpha=0.00001), X_st)
y_pred_proba

100000 [ 0.25809285 -0.68188567  0.6883469   1.2412009 ] 0.6348708150530281


array([0.5357167 , 0.60516312, 0.74217791, 0.57078825, 0.79724039,
       0.77745014, 0.78877271, 0.20067225, 0.51802515, 0.8041147 ])

In [11]:
def calc_pred(W, X):
    m = X.shape[1]
    
    y_pred = np.zeros((1, m))
    W = W.reshape(X.shape[0], 1)
    
    A = sigmoid(np.dot(W.T, X))
    
    for i in range(A.shape[1]):
        if (A[:,i] > 0.5): 
            y_pred[:, i] = 1
        elif (A[:,i] <= 0.5):
            y_pred[:, i] = 0

    return y_pred

In [14]:
y_pred = calc_pred(eval_model(X_st, y, iterations=n_iterations, alpha=0.00001), X_st)
y_pred

100000 [ 0.25809285 -0.68188567  0.6883469   1.2412009 ] 0.6348708150530281


array([[1., 1., 1., 1., 1., 1., 1., 0., 1., 1.]])

In [18]:
tp = 0
tn = 0
fp = 0
fn = 0

In [28]:
for i in range(len(y)):
    if y[i] == 1 and y_pred[0][i] == 1: 
        tp += 1
    elif y[i] == 1 and y_pred[0][i] == 0: 
        fn += 1
    elif y[i] == 0 and y_pred[0][i] == 1: 
        fp += 1
    else: 
        tn += 1

In [30]:
confusion_matrix = np.array([[tp, fp], [fn, tn]])
confusion_matrix

array([[5, 4],
       [0, 1]])

In [31]:
accuracy = (tp + tn) / (tp + tn + fp + fn)
accuracy

0.6

In [32]:
precision = tp / (tp + fp)
precision

0.5555555555555556

In [33]:
recall = tp / (tp + fn)
recall

1.0

In [34]:
f_score = 2 * precision * recall / (precision + recall)
f_score

0.7142857142857143